<a href="https://colab.research.google.com/github/CarOmodeo/practicos_pnl/blob/main/Desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

'00072741'

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [ ]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))


**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.


In [ ]:
#vectorización de los documentos
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

y_train = newsgroups_train.target

In [ ]:
#Obtenemos 5 textos al azar para comparar
#la similitus de estos con el resto de los documentos

random_idx = np.random.choice(len(newsgroups_train.data), 5, replace=False)
print(random_idx)

[11132  1429  4315  9093  6586]


In [ ]:
# TEXTO 1
print("Text N°", random_idx[0])
print(newsgroups_train.data.__getitem__(random_idx[0]))
print("Theme: ",newsgroups_train.target_names[y_train[random_idx[0]]])

#similitud con el resto de los textos
cossim = cosine_similarity(X_train[random_idx[0]], X_train)[0]

# Obtenemos los 5 textos con mayores similitudes
mostsim = np.argsort(cossim)[::-1][1:6]
mostsim_texts = [newsgroups_train.data.__getitem__(j) for j in mostsim]

print("\nSIMILAR TEXTS:")
for k in mostsim:
  print("Text N°", k)
  print(newsgroups_train.data.__getitem__(k))
  print("Theme: ",newsgroups_train.target_names[y_train[k]], "\n------------------------\n")

Text N° 11132





GOT HIM!  Cramer is now claiming that pedophilia is a sexual orientation
rather than a chronicly homosexual condition.  This changes the whole
argument in as much that is pedophilia is a sexual orientation all
of its own peds cannot be called homosexual.  Peds are peds who
may have a preference as to the sex of the child they molest (though
most do not have a preference) but that is a subset of their basic
ped nature.

Cramer has as much as admitted that peds and gay men are different
orientations.  All we need now is to get him to admit that
the apparent similarities he keeps on about are just optical illusions.

xavier
Theme:  talk.politics.misc

SIMILAR TEXTS:
Text N° 5159

Please point out where I have said I even _was_ a member of that group,
much less asked to be treated as such, much less insisted upon it.


Wait a minute.  You've been claiming for quite a while now that
pedophilia, according to CA state law, is a sexual orientation.  Now
your position is that

In [ ]:
# TEXTO 2
print("Text N°", random_idx[1])
print(newsgroups_train.data.__getitem__(random_idx[1]))
print("Theme: ",newsgroups_train.target_names[y_train[random_idx[1]]])

#similitud con el resto de los textos
cossim = cosine_similarity(X_train[random_idx[1]], X_train)[0]

# Obtenemos los 5 textos con mayores similitudes
mostsim = np.argsort(cossim)[::-1][1:6]
mostsim_texts = [newsgroups_train.data.__getitem__(j) for j in mostsim]

print("\nSIMILAR TEXTS:")
for k in mostsim:
  print("Text N°", k)
  print(newsgroups_train.data.__getitem__(k))
  print("Theme: ",newsgroups_train.target_names[y_train[k]], "\n------------------------\n")

Text N° 1429
Can anyone help me find any information on the drug Prozac?  I am writing
a report on the inventors , Eli Lilly and Co., and the product.  I need as
much help as I can get.   Thanks a lot, Adriana Gilmete.

Theme:  sci.med

SIMILAR TEXTS:
Text N° 1710

 Well, yes.  That was a part of my point.  Aspirin has its problems, but
in some situations it is useful.  Ditto stuff like licorice root.  Taking
anything as a drug for theraputic purposes implicitly carries the idea
of taking a dose where the benefits are not exceeded by any unwanted,
additional effects.  Taking any drug when the potential ill-effects are
not known is a risk assumed by the parties involved, and it may be that
in a given situation the risk is worthwhile.
   Like Prozac, for instance; Prozac has been shown to be theraputic in
some cases where the tri-cyclics fail.  But Prozac hasn't been in use
that long, and it really isn't clear what if any effects it may have
when taken over long periods of time, even tho

In [ ]:
# TEXTO 3
print("Text N°", random_idx[2])
print(newsgroups_train.data.__getitem__(random_idx[2]))
print("Theme: ",newsgroups_train.target_names[y_train[random_idx[2]]])

#similitud con el resto de los textos
cossim = cosine_similarity(X_train[random_idx[2]], X_train)[0]

# Obtenemos los 5 textos con mayores similitudes
mostsim = np.argsort(cossim)[::-1][1:6]
mostsim_texts = [newsgroups_train.data.__getitem__(j) for j in mostsim]

print("\nSIMILAR TEXTS:")
for k in mostsim:
  print("Text N°", k)
  print(newsgroups_train.data.__getitem__(k))
  print("Theme: ",newsgroups_train.target_names[y_train[k]], "\n------------------------\n")

Text N° 4315

Theme:  talk.politics.mideast

SIMILAR TEXTS:
Text N° 3712


Again, from my alcohol server's class:
The absolute *most* that eating before drinking can do is slow the absorption
down by 15 minutes.  That gives me time to eat, slam one beer, and ride like
hell to try to make it home in the 10 minutes left after paying, donning 
helmet & gloves, starting bike...


Theme:  rec.motorcycles 
------------------------

Text N° 3776
I will try to answer some of Dorin's questions, even though they were
not addressed to me specifically, but I feel that I am a bit concerned
by the thread since I am a Southern Lebanese from a village that is 
often on the receiving end of Israel's bombs.
In the first place the death of three soldiers on a patrol in occupied
Lebanese terrritory is NOT an act of terrorism or murder.  It is 
disingeneous to compare their death to that of athletes in Munich
or any other act of terrorism or mrder.  This exercise is aimed 
solely at diverting the issue and

In [ ]:
# TEXTO 4
print("Text N°", random_idx[3])
print(newsgroups_train.data.__getitem__(random_idx[3]))
print("Theme: ",newsgroups_train.target_names[y_train[random_idx[3]]])

#similitud con el resto de los textos
cossim = cosine_similarity(X_train[random_idx[3]], X_train)[0]

# Obtenemos los 5 textos con mayores similitudes
mostsim = np.argsort(cossim)[::-1][1:6]
mostsim_texts = [newsgroups_train.data.__getitem__(j) for j in mostsim]

print("\nSIMILAR TEXTS:")
for k in mostsim:
  print("Text N°", k)
  print(newsgroups_train.data.__getitem__(k))
  print("Theme: ",newsgroups_train.target_names[y_train[k]], "\n------------------------\n")

Text N° 9093
In-reply-to: todamhyp@charles.unlv.edu's message of Mon, 19 Apr 93 20:56:15 GMT
Newsgroups: sci.energy,sci.image.processing,sci.anthropology,alt.sci.physics.new-theories,sci.skeptic,sci.med,alt.alien.visitors
Subject: Re: Krillean Photography
References: <1993Apr19.205615.1013@unlv.edu>
Distribution: 
--text follows this line--

	   I am looking for any information/supplies that will allow
   do-it-yourselfers to take Krillean Pictures. I'm thinking
   that education suppliers for schools might have a appartus for
   sale, but I don't know any of the companies. Any info is greatly
   appreciated.
	   In case you don't know, Krillean Photography, to the best of my
   knowledge, involves taking pictures of an (most of the time) organic
   object between charged plates. The picture will show energy patterns
   or spikes around the object photographed, and depending on what type
   of object it is, the spikes or energy patterns will vary. One might
   extrapolate here and say 

In [ ]:
# TEXTO 5
print("Text N°", random_idx[4])
print(newsgroups_train.data.__getitem__(random_idx[4]))
print("Theme: ",newsgroups_train.target_names[y_train[random_idx[4]]])

#similitud con el resto de los textos
cossim = cosine_similarity(X_train[random_idx[4]], X_train)[0]

# Obtenemos los 5 textos con mayores similitudes
mostsim = np.argsort(cossim)[::-1][1:6]
mostsim_texts = [newsgroups_train.data.__getitem__(j) for j in mostsim]

print("\nSIMILAR TEXTS:")
for k in mostsim:
  print("Text N°", k)
  print(newsgroups_train.data.__getitem__(k))
  print("Theme: ",newsgroups_train.target_names[y_train[k]], "\n------------------------\n")

Text N° 6586

<<<most of message deleted>>>


Theme:  sci.space

SIMILAR TEXTS:
Text N° 4739
Stuff deleted...
Theme:  rec.autos 
------------------------

Text N° 6616
stuff deleted...
Theme:  rec.autos 
------------------------

Text N° 6366
//// Much stuff deleted //////
Theme:  comp.os.ms-windows.misc 
------------------------

Text N° 3973
It's only test message.


Theme:  rec.sport.hockey 
------------------------

Text N° 2419
Jim Brown wrote :

[ deleted ]
[ deleted ]

  Unfortunately, it's not that simple.  The KJV is preferred by the majority
of fundamentalists (at least here).  The second part of your argument fails
as well, since that statement can be used against any version (not just the
KJV).

[ deleted ]
[ deleted ]

  I would not find this statement to be very useful since it is an appeal
to authority and the opposition will just claim that their authorities are
"better".  A second tact that local creationists have used is to reply "but
those scholars are atheists and c

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups

In [ ]:
# Cargar datos
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

X_train, y_train = newsgroups_train.data, newsgroups_train.target
X_test, y_test = newsgroups_test.data, newsgroups_test.target

In [ ]:
# Definir una pipeline con el vectorizador y el modelo
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', ComplementNB())  # Inicialmente MultinomialNB
])

In [ ]:
# Definir los parámetros a probar
param_grid = {
   'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'vectorizer__max_df': [0.5, 0.7, 0.9],
    'vectorizer__min_df': [1, 2, 5],
    'vectorizer__use_idf': [True, False],
    'vectorizer__sublinear_tf': [True, False],
    'classifier__alpha': [0.01, 0.1, 1],
    'classifier__fit_prior': [True, False]
}

In [ ]:
# Usar GridSearchCV para buscar la mejor combinación de parámetros
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                                       ('classifier', ComplementNB())]),
             n_jobs=-1,
             param_grid={'classifier__alpha': [0.01, 0.1, 1],
                         'classifier__fit_prior': [True, False],
                         'vectorizer__max_df': [0.5, 0.7, 0.9],
                         'vectorizer__min_df': [1, 2, 5],
                         'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
                         'vectorizer__sublinear_tf': [True, False],
                         'vectorizer__use_idf': [True, False]},
             scoring='f1_macro')

In [ ]:
# Mostrar los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

Mejores parámetros: {'classifier__alpha': 0.1, 'classifier__fit_prior': True, 'vectorizer__max_df': 0.9, 'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 2), 'vectorizer__sublinear_tf': True, 'vectorizer__use_idf': True}


In [ ]:
# Evaluar en el conjunto de test
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_f1_macro = f1_score(y_test, y_pred, average='macro')
print("F1-Score macro en test:", test_f1_macro)

F1-Score macro en test: 0.7059049460930455


# Conclusión

El modelo optimizado con ComplementNB y vectorización TF-IDF logró un F1-Score macro de 0.706, reflejando un buen rendimiento en la clasificación balanceada de las categorías del conjunto de datos de prueba. Los ajustes realizados en los parámetros del vectorizador y el clasificador permitieron capturar mejor las relaciones entre términos y documentos, mostrando que el enfoque es adecuado para el problema planteado. Sin embargo, aún existe margen de mejora mediante técnicas adicionales de preprocesamiento o modelos más avanzados.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups

# Cargar datos
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

X_train, y_train = newsgroups_train.data, newsgroups_train.target
X_test, y_test = newsgroups_test.data, newsgroups_test.target

# Vectorización de los documentos (documento-término)
tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [ ]:
# Transponer la matriz para obtener la matriz término-documento
X_train_t = X_train.T

In [ ]:
# Obtener todas las palabras del vocabulario
words = tfidfvect.get_feature_names_out()

# Elegir manualmente 5 palabras de interés
chosen_words = ["computer", "politics", "sports", "technology", "government"]
chosen_indices = [words.tolist().index(word) for word in chosen_words]

In [ ]:
# Mostrar las palabras más similares para cada palabra seleccionada
for idx, word_idx in enumerate(chosen_indices):
    # Similaridad de la palabra actual con todo el vocabulario
    similarities = cosine_similarity(X_train_t[word_idx], X_train_t)[0]

    # Ordenar por similitud (excluyendo la palabra consigo misma)
    similar_indices = similarities.argsort()[-6:-1][::-1]

    print(f"Palabras más similares a '{words[word_idx]}':")
    for sim_idx in similar_indices:
        print(f"- {words[sim_idx]} (Similitud: {similarities[sim_idx]:.4f})")
    print("\n")

Palabras más similares a 'computer':
- decwriter (Similitud: 0.1563)
- harkens (Similitud: 0.1522)
- deluged (Similitud: 0.1522)
- shopper (Similitud: 0.1443)
- the (Similitud: 0.1361)


Palabras más similares a 'politics':
- iftccu (Similitud: 0.3139)
- hesh (Similitud: 0.2634)
- fascism (Similitud: 0.2599)
- bmwmoa (Similitud: 0.2548)
- lapse (Similitud: 0.2395)


Palabras más similares a 'sports':
- wip (Similitud: 0.3699)
- rockin (Similitud: 0.3599)
- pollute (Similitud: 0.3599)
- jockeys (Similitud: 0.3599)
- admittdly (Similitud: 0.2565)


Palabras más similares a 'technology':
- blatently (Similitud: 0.3101)
- lecturing (Similitud: 0.3101)
- ellul (Similitud: 0.3101)
- christan (Similitud: 0.3101)
- toffler (Similitud: 0.3101)


Palabras más similares a 'government':
- the (Similitud: 0.2410)
- to (Similitud: 0.2251)
- of (Similitud: 0.2227)
- libertarian (Similitud: 0.2199)
- encryption (Similitud: 0.2174)


